In [ ]:
# Import and init h2o lib. Running on java
import h2o
from h2o.automl import H2OAutoML
import pandas as pd
h2o.init()

In [ ]:
# Reading csv to h2o dataframe 
col_types_train = {'store_id': 'enum',
 'chain_name': 'enum',
 'mall_name': 'enum',
 'lv1': 'enum',
 'lv2': 'enum',
 'lv3': 'enum',
 'revenue': 'int',
 'total_nbr_people': 'int',
 'nbr_people_per_store_in_grunnkrets': 'int',
 'counts_gr_lv2': 'int',
 'counts_municipality_lv2': 'int',
 'busstops_within_50m': 'int',
 'busstops_within_100m': 'int',
 'busstops_within_400m': 'int',
 'busstops_within_800m': 'int',
 'busstops_within_1500m': 'int',
 'num_stores_within_100m_and_same_lvl2': 'int',
 'num_stores_within_500m_and_same_lvl2': 'int',
 'num_stores_within_1km_and_same_lvl2': 'int',
 'num_stores_within_5km_and_same_lvl2': 'int',
 'num_stores_within_10km_and_same_lvl2': 'int',
 'num_stores_within_20km_and_same_lvl2': 'int',
 'municipality_size_group': 'enum',
 'mean_revenue_for_municipality_size_group': 'int',
 'median_revenue_for_municipality_size_group': 'int',
 'st_dev_of_revenue_for_municipality_size_group': 'int'}

col_types_test = {'store_id': 'enum',
 'chain_name': 'enum',
 'mall_name': 'enum',
 'lv1': 'enum',
 'lv2': 'enum',
 'lv3': 'enum',
 'total_nbr_people': 'int',
 'nbr_people_per_store_in_grunnkrets': 'int',
 'counts_gr_lv2': 'int',
 'counts_municipality_lv2': 'int',
 'busstops_within_50m': 'int',
 'busstops_within_100m': 'int',
 'busstops_within_400m': 'int',
 'busstops_within_800m': 'int',
 'busstops_within_1500m': 'int',
 'num_stores_within_100m_and_same_lvl2': 'int',
 'num_stores_within_500m_and_same_lvl2': 'int',
 'num_stores_within_1km_and_same_lvl2': 'int',
 'num_stores_within_5km_and_same_lvl2': 'int',
 'num_stores_within_10km_and_same_lvl2': 'int',
 'num_stores_within_20km_and_same_lvl2': 'int',
 'municipality_size_group': 'enum',
 'mean_revenue_for_municipality_size_group': 'int',
 'median_revenue_for_municipality_size_group': 'int',
 'st_dev_of_revenue_for_municipality_size_group': 'int'}

 
train_data = h2o.import_file('feature_data/training_set_dropped.csv', col_types = col_types_train)
test_data = h2o.import_file('feature_data/testing_set_dropped.csv', col_types = col_types_test)

test = pd.read_csv('feature_data/testing_set_dropped.csv')

# Save store ids, then remove this column
store_ids = test['store_id'].to_numpy()
ids = pd.DataFrame(store_ids)
ids.rename(columns={0 :'id'}, inplace=True )

train_data.types



In [ ]:
# Removing labels from train set
y = "revenue"
x = train_data.columns
x.remove(y)

In [ ]:
# Running fit/training on train set with cross validation n=5
aml = H2OAutoML(max_models = 20, seed = 1)
aml.train(x = x, y = y, training_frame = train_data)

In [ ]:
lb = aml.leaderboard

In [110]:
# Showing best perforimg models
lb.head()

model_id,rmse,mse,mae,rmsle,mean_residual_deviance
StackedEnsemble_AllModels_1_AutoML_4_20221109_120957,0.666238,0.443873,0.501055,0.283,0.443873
StackedEnsemble_BestOfFamily_1_AutoML_4_20221109_120957,0.666982,0.444865,0.50236,0.283456,0.444865
GBM_1_AutoML_4_20221109_120957,0.672445,0.452182,0.506048,0.284425,0.452182
GBM_2_AutoML_4_20221109_120957,0.67291,0.452808,0.505471,0.284487,0.452808
GBM_3_AutoML_4_20221109_120957,0.673476,0.453569,0.505871,0.284555,0.453569
DRF_1_AutoML_4_20221109_120957,0.678003,0.459688,0.508847,0.286617,0.459688
GBM_4_AutoML_4_20221109_120957,0.678658,0.460577,0.509759,0.286496,0.460577
XGBoost_3_AutoML_4_20221109_120957,0.710814,0.505257,0.54644,0.300959,0.505257
GLM_1_AutoML_4_20221109_120957,0.72284,0.522498,0.559994,0.304878,0.522498
XRT_1_AutoML_4_20221109_120957,0.725354,0.526138,0.567141,0.30309,0.526138


In [ ]:
preds = aml.predict(test_data)

In [ ]:
#preds = h2o.as_list(preds)
print(preds)

In [ ]:
# Convert result to desired format

import pandas as pd
import numpy as np


pandas_preds = preds.as_data_frame()
pandas_preds.rename(columns={'predict' :'predicted'}, inplace=True)
pandas_preds['predicted'] = np.expm1(pandas_preds['predicted'])

def save_submission(pred):
    predicted = pd.DataFrame(pred, columns = ['predicted'])
    output = pd.concat([ids,predicted],axis=1)
    output.to_csv('h2o.csv', index=False)
    
save_submission(pandas_preds)

